# More Data

In the prior example [Benefits](1 - Benefits.ipynb), examples were shown that simply retrieved a single timestep.

Specifically, this was done with a fully resolved specification of time, the data indexes had a temporal resolution of hourly, and then full hour specification was given.

This example will show more details on how that works, and how to retrieve a time series of data.

In [ ]:
import edit.data

In [2]:
import xarray as xr
xr.set_options(display_expand_attrs = False);

In [3]:
ERA5_index = edit.data.archive.NCI.ERA5('tcwv')
ERA5_index

ERA5
	Description                    ECWMF ReAnalysis v5
		 range                          1970-current
		 Documentation                  https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation


	Initialisation                 
		 level_value                    None
		 product                        reanalysis
		 transforms                     {}
		 variables                      ['tcwv']
	Transforms                     
		 ConformNaming                  {'latitude': ['lat', 'Latitude', 'yt_ocean', 'yt'], 'longitude': ['lon', 'Longitude', 'xt_ocean', 'xt'], 'time': ['Time']}
		 Rename                         {'t2m': '2t', 'u10': '10u', 'v10': '10v', 'siconc': 'ci'}

## Resolution Aware
As mentioned, the `edit.data.Indexes` includes a resolution aware datetime implementation, and thus data can be retrieved according to the resolution of the time given.

Say for data with hourly data, the following behaviours can be expected
| Date      | Behaviour               |
| --------- | ----------------          |
|`2021-01-01T00:00`|Exact Data            |
|`2021-01-01T00`   |All Data in that hour  |
|`2021-01-01`      |All Data in that day  |
|`2021-01`         |All Data in that month|
|`2021`            |All Data in that year |
        
Therefore, by overridding the resolution of the datetime, different sets of the data can be retrieved.

In [4]:
time_of_interest = edit.data.EDITDatetime('2022-01-01T00')
time_of_interest

EDITDatetime('2022-01-01T00')

In [5]:
for resolution in edit.data.time.RESOLUTION_COMPONENTS[::-1]:
    print(resolution, '\t\t', time_of_interest.at_resolution(resolution))

nanosecond 		 2022-01-01T00:00:00.0
second 		 2022-01-01T00:00:00
minute 		 2022-01-01T00:00
hour 		 2022-01-01T00
day 		 2022-01-01
month 		 2022-01
year 		 2022


In [6]:
# Full specification of time
ERA5_index[time_of_interest.at_resolution('hour')]

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 1)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2022-01-01
Data variables:
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(1, 182, 360), meta=np.ndarray>
Attributes: (5)

In [7]:
# Day specification
ERA5_index[time_of_interest.at_resolution('day')]

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 24)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-01-01T23:00:00
Data variables:
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(24, 182, 360), meta=np.ndarray>
Attributes: (5)

In [8]:
# Month specification
ERA5_index[time_of_interest.at_resolution('month')]

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 744)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-01-31T23:00:00
Data variables:
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(186, 182, 360), meta=np.ndarray>
Attributes: (5)

In [9]:
# Year specification
ERA5_index[time_of_interest.at_resolution('year')]

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 8760)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-12-31T23:00:00
Data variables:
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(186, 182, 360), meta=np.ndarray>
Attributes: (3)

## Series of data
If the data needed does not fit within a resolution bound, it is easy to set your own bounds.

`.series` takes the same style of arguments as `range` does, with a start, end & interval. 
Additionally, interval is optional, with it defaulting to the resolution of the data itself.

Let's get 10 days of data from ERA5 at 6 hourly interval.

In [10]:
ERA5_index.series(time_of_interest, time_of_interest + edit.data.TimeDelta(10, 'days'), edit.data.TimeDelta(6, 'hours'))

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 40)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-01-10T18:00:00
Data variables:
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(31, 182, 360), meta=np.ndarray>
Attributes: (5)